# Network Analysis

In [3]:
import os
import numpy as np
import powerlaw

import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
FONTSIZE=16
LABELSIZE=0.9*FONTSIZE
LINEWIDTH=2.0
FIGWIDTH=8
ASPECT_RATIO=4/3

plt.rcParams['axes.labelsize'] = FONTSIZE
plt.rcParams['axes.linewidth'] = LINEWIDTH
plt.rcParams['axes.titlesize'] = 1.1*FONTSIZE
plt.rcParams['figure.figsize'] = [FIGWIDTH, FIGWIDTH/ASPECT_RATIO]
plt.rcParams['figure.labelsize'] = FONTSIZE
plt.rcParams['figure.titlesize'] = 1.1*FONTSIZE
plt.rcParams['legend.fancybox'] = False
plt.rcParams['legend.framealpha'] = 1.0
plt.rcParams['legend.fontsize'] = LABELSIZE
plt.rcParams['legend.title_fontsize'] = LABELSIZE
plt.rcParams['patch.linewidth'] = LINEWIDTH
plt.rcParams['lines.linewidth'] = LINEWIDTH
plt.rcParams['xtick.labelsize'] = LABELSIZE
plt.rcParams['ytick.labelsize'] = LABELSIZE
plt.rcParams['xtick.minor.visible'] = True
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['figure.autolayout'] = True

In [58]:
# Carica il network completo
full_graph_backbone = nx.read_gexf(os.path.join("..","data_collection","graph","full_network_backbone_opinion.gexf"))

In [42]:
# Estrae il grafico Undirected e Unweighted
real_graph = full_graph_backbone.to_undirected()

# Rimuovere l'attributo "weight" da tutti gli archi
for u, v in real_graph.edges():
    if 'weight' in real_graph[u][v]:  # Verifica se esiste il peso
        del real_graph[u][v]['weight']

In [53]:
# Il grafico non è più diretto
for G in [full_graph_backbone, real_graph]:
    print(f"E' diretto? {G.is_directed()}")

E' diretto? True
E' diretto? False


In [59]:
# Il grafico non ha più l'attributo peso
u = 'Sufficient-Cat-5399'
v = 'Hesfallenontheice'
for G in [full_graph_backbone, real_graph]:
    if G.has_edge(u, v):
        edge_data = G[u][v]  # Ottiene gli attributi dell'edge (come dizionario)
        print(f"Edge ({u}, {v}):")
        try: print(f"    Weight: {edge_data['weight']}\n")
        except KeyError: print("    L'attributo 'weight' non è presente")

Edge (Sufficient-Cat-5399, Hesfallenontheice):
    Weight: 2.0

Edge (Sufficient-Cat-5399, Hesfallenontheice):
    L'attributo 'weight' non è presente


In [54]:
print("Weigthed and Directed")
print(f"Nodi: {full_graph_backbone.number_of_nodes()}, Archi: {full_graph_backbone.number_of_edges()}")

print("\nUnweigthed and Undirected:")
print(f"Nodi: {real_graph.number_of_nodes()}, Archi: {real_graph.number_of_edges()}")

Weigthed and Directed
Nodi: 18941, Archi: 72726

Unweigthed and Undirected:
Nodi: 18941, Archi: 66214


Il **numero di archi diminuisce** perché, nel caso in cui due nodi sono collegati con un arco entrante ed uno uscente, questi due archi vengono collassati in un unico collegamento senza direzione.

### Caratteristiche grafico 

In [15]:
N = real_graph.number_of_nodes()
L = real_graph.number_of_edges()
L_max = N*(N-1)/2

In [14]:
p = L/L_max

if p > np.log(N)/N: print(f"p = {p} > log(N)/N = {np.log(N)/N}  =>  Connected")
elif p > 1/ N: print(f"p = {p} > 1/N = {1/N}  =>  Supercritical")
elif p == 1/ N: print(f"p = {p} = 1/N = {1/N}  =>  Critical")
elif p < 1/N: print(f"p = {p} < 1/N = {1/N}  =>  Subcritical")


p = 0.00036914495838714863 > 1/N = 5.279552293965472e-05  =>  Supercritical


In [24]:
print(f"Number of component: {nx.number_connected_components(real_graph)}")
size_principal_component = len(list(nx.connected_components(real_graph))[0])
print(f"Principal component: {size_principal_component/N:.2f}% ({size_principal_component}/{N})")

Number of component: 216
Principal component: 0.98% (18485/18941)


In [60]:
print(f"Average degree: {sum(dict(real_graph.degree()).values())/float(len(real_graph))}")
print(f"Global clustering: {nx.average_clustering(real_graph)}")

# Tempo di coputazione elevato => Average shortest path: 4.659564546301864
principal_component = nx.subgraph(real_graph, list(nx.connected_components(real_graph))[0])
#print(f"Average shortest path: {nx.average_shortest_path_length(principal_component)}")

Average degree: 6.991605511852595
Global clustering: 0.042378415673978265
Average shortest path: 4.659564546301864


## Creazione grafici sintetici

In [ ]:
# Random
random_graph = nx.erdos_renyi_graph(N, L/L_max)  # p = L/L_max

In [ ]:
# Small world
